#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys

# 将上级目录加入系统路径
sys.path.append(os.path.abspath('..'))

from src.utils import load_multiver_corpus, save_results
from src.annotator.semantic_annotation import SemanticAnnotator, annotate_data

#### **2. 读取语料**

读取标注词性的多版本译文平行语料库

In [2]:
# 指定数据文件路径
data_file = '../data/output/1_mt_generation.jsonl'
print(f"从 {os.path.abspath(data_file)} 文件读取数据 ...")

# 读取数据：
data = load_multiver_corpus(data_file)
print(f"成功读取 {len(data)} 条平行句对")

# 预览数据：
# 导入后的数据以 DataFrame 格式存储
# 第一列：原文
# 第二列：多版本译文：
# human：人类译本（闵福德译）
# deepseek-v3.2：deepseek 译本
# qwen3-max：qwen 译本

print("数据前 5 行如下：")
data.head()

从 C:\llm_corpus_annotation_v3\data\output\1_mt_generation.jsonl 文件读取数据 ...
成功读取 300 条平行句对
数据前 5 行如下：


,id,source,targets
0,1,{'raw_text': '江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒...,{'human': {'raw_text': 'Along a coastal road s...
1,2,{'raw_text': '前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者...,{'human': {'raw_text': 'In each of the first t...
2,3,{'raw_text': '后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。'},{'human': {'raw_text': 'The four rear carts we...
3,4,{'raw_text': '女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。'},{'human': {'raw_text': 'The little girl was cr...
4,5,{'raw_text': '囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！'},{'human': {'raw_text': 'One of the soldiers ma...


In [3]:
# 平行语料原文 source 节选自《鹿鼎记》
# 译文 targets 包括 3 个版本：
# human：人类译本（闵福德译）
# deepseek-v3.2：deepseek 译本
# qwen3-max：qwen 译本

# 预览数据
print(f"数据前 5 行如下：")
data.head()

数据前 5 行如下：


,id,source,targets
0,1,{'raw_text': '江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒...,{'human': {'raw_text': 'Along a coastal road s...
1,2,{'raw_text': '前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者...,{'human': {'raw_text': 'In each of the first t...
2,3,{'raw_text': '后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。'},{'human': {'raw_text': 'The four rear carts we...
3,4,{'raw_text': '女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。'},{'human': {'raw_text': 'The little girl was cr...
4,5,{'raw_text': '囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！'},{'human': {'raw_text': 'One of the soldiers ma...


#### **3. 加载模型**

加载 LLM 语义标注模型

In [4]:
# 指定用于语义标注的大模型
# 可从以下模型选取：
# qwen-flash, qwen-plus, qwen3-max, glm-4.7, deepseek-v3.2
model = 'glm-4.7'

# 指定语种
# 可选语种：English，Chinese
lang = 'English'

# 加载模型前，请登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账户
# 并在 llm_corpus_annotation/.env 文件中设置 LLM_API_KEY=sk-********

# 加载中文语义标注模型
#zh_annotator = SemanticAnnotator(lang='Chinese', model=model)
#print(f"中文语义标注模型加载完毕：{zh_annotator.model}")

# 加载英文语义标注模型
annotator = SemanticAnnotator(lang=lang, model=model)
print("语义标注模型加载完毕！")
print(f"模型：{annotator.model}")
print(f"语种：{annotator.lang}")

语义标注模型加载完毕！
模型：glm-4.7
语种：English


#### **4. 批量标注**

标注中文英文各译本的语义类别，并以 JSON 格式返回标注结果

In [5]:
# 注意：
# 为节省 API 调用成本，本程序将大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，程序运行时间显示为 0.0 秒

# 开始标注
results = annotate_data(data, annotator)

Semantic Tagging: 100%|████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 5285.54it/s]


#### **5. 保存结果**

将大模型译文保存于指定的文件中

In [6]:
# 指定输出文件
out_file = f"../data/output/3_semantic_annotation.jsonl"

# 保存标注结果
save_results(results, out_file)
print(f"结果已保存至 {out_file}\n")

结果已保存至 ../data/output/3_semantic_annotation.jsonl



#### **6. 查看结果**

预览英文标注结果，查看语义赋码集

In [7]:
# 预览前数据
for res in results[:1]:
    # 提取数据
    source = res['source']
    targets = res['targets']

    print(f"\n[ID]: {res['id']}")

    # 中文
    src_text = source['raw_text']
    print(f"{src_text}")
    print("-" * 60)

    # 英文
    for version, anno in targets.items():
        if not anno: continue
        print(f"[{version.upper()}]")
        
        tgt_text =anno['raw_text']
        print(f"{tgt_text}\n")
        usas_list = anno['usas_tags'][:5]
        for tags in usas_list:
            print(f"Text: {tags['text']:<15}USAS: {tags['tag']:<15}DESC: {tags['desc']:<15}")
        print("......")
        print("=" * 60)


[ID]: 000000
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
------------------------------------------------------------
[HUMAN]
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.

Text: Along          USAS: M6             DESC: Location and direction
Text: a coastal road USAS: M3             DESC: Vehicles and transport on land
Text: somewhere      USAS: M7             DESC: Places         
Text: south          USAS: M6             DESC: Location and direction
Text: of             USAS: Z5             DESC: Grammatical bin
......
[DEEPSEEK-V3.2]
Along a broad road near the coast south of the Yangtze River, a troop of Qing soldiers, armed with swords and spears, escorted seven prison carts northward, braving the biting wind and cold.

Text: Along          USAS: M6             DESC: Location and direction
Text: a              USAS: 